# BLG-407 Makine Öğrenmesi – Proje 2  
## YOLOv8 ile Kedi / Köpek Nesne Tespiti ve PyQt5 Tabanlı Görsel Arayüz

**Ad:** İbrahim  
**Soyad:** Çerkezoğlu  
**Okul No:** 2112721046  

**GitHub Repo Bağlantısı:** https://github.com/ibrahimcerkezoglu/YOLOv8_CatDog_Detection  

Bu notebook’ta, kedi ve köpek sınıflarından oluşan kendi veri setim üzerinde YOLOv8 nesne tespiti modeli eğitilmekte ve elde edilen `best.pt` modeli PyQt5 tabanlı bir GUI uygulamasında kullanılmaktadır.


## 1. Veri Seti Hazırlığı

- Sınıflar: **cat**, **dog** (toplam 2 sınıf)  
- Train görüntü sayısı: **462**  
- Valid görüntü sayısı: **53**  
- Test görüntü sayısı: **22**  
- Etiketleme formatı: YOLOv8 uyumlu `.txt` dosyaları  
- Klasör yapısı: 
dataset/
├── train/
│ ├── images/
│ └── labels/
├── valid/
│ ├── images/
│ └── labels/
└── test/
├── images/
└── labels/



In [1]:
from ultralytics import YOLO
import os
import cv2
import matplotlib.pyplot as plt
from pathlib import Path
import torch

print("PyTorch sürümü:", torch.__version__)
print("CUDA aktif mi:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))


PyTorch sürümü: 2.5.1+cu121
CUDA aktif mi: True
GPU: NVIDIA GeForce RTX 4060 Laptop GPU


In [2]:
# Çalışma dizini (notebook'un açıldığı klasör)
PROJECT_DIR = Path().resolve()
print("Çalışma klasörü:", PROJECT_DIR)

# data.yaml yolu
DATA_YAML = PROJECT_DIR / "dataset" / "data.yaml"
print("DATA_YAML:", DATA_YAML)
assert DATA_YAML.exists(), "dataset/data.yaml dosyası bulunamadı!"

# Dataset klasörlerini kontrol edelim
train_images = PROJECT_DIR / "dataset" / "train" / "images"
valid_images = PROJECT_DIR / "dataset" / "valid" / "images"
test_images = PROJECT_DIR / "dataset" / "test" / "images"

print("Train image sayısı:", len(os.listdir(train_images)))
print("Valid image sayısı:", len(os.listdir(valid_images)))
print("Test  image sayısı:", len(os.listdir(test_images)))


Çalışma klasörü: C:\Users\ibrah\Documents\BLG407_YOLOv8_Proje
DATA_YAML: C:\Users\ibrah\Documents\BLG407_YOLOv8_Proje\dataset\data.yaml
Train image sayısı: 462
Valid image sayısı: 53
Test  image sayısı: 22


## 2. YOLOv8 Modelinin Yüklenmesi ve Eğitimi


In [3]:
MODEL_NAME = "yolov8n.pt"  # pre-trained COCO modeli

# Modeli yükle (yoksa otomatik indirir)
model = YOLO(MODEL_NAME)
model  # mimari özetini gösterir


YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

In [4]:
results = model.train(
    data=str(DATA_YAML),   # data.yaml yolu
    epochs=30,             # istersen 50'ye çıkarırsın
    imgsz=640,
    batch=8,
    name="cats_dogs_v1",   # run ismi
    project="runs/train",  # runs/train/cats_dogs_v1
    patience=10            # erken durdurma sabrı
)


New https://pypi.org/project/ultralytics/8.3.235 available  Update with 'pip install -U ultralytics'
Ultralytics 8.3.0  Python-3.10.19 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
engine\trainer: task=detect, mode=train, model=yolov8n.pt, data=C:\Users\ibrah\Documents\BLG407_YOLOv8_Proje\dataset\data.yaml, epochs=30, time=None, patience=10, batch=8, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=runs/train, name=cats_dogs_v12, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False,

100%|██████████| 755k/755k [00:00<00:00, 2.30MB/s]


Overriding model.yaml nc=80 with nc=2

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

train: Scanning C:\Users\ibrah\Documents\BLG407_YOLOv8_Proje\dataset\train\labels... 462 images, 0 backgrounds, 0 corrupt: 100%|██████████| 462/462 [00:01<00:00, 431.16it/s]


train: New cache created: C:\Users\ibrah\Documents\BLG407_YOLOv8_Proje\dataset\train\labels.cache


...

      30/30      1.21G     0.9027     0.9387      1.319          9        640: 100%|██████████| 58/58 [00:04<00:00, 13.69it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00, 13.09it/s]

                   all         53         95      0.443      0.584       0.51      0.308



30 epochs completed in 0.051 hours.
Optimizer stripped from runs\train\cats_dogs_v12\weights\last.pt, 5.6MB
Optimizer stripped from runs\train\cats_dogs_v12\weights\best.pt, 5.6MB

Validating runs\train\cats_dogs_v12\weights\best.pt...
Ultralytics 8.3.0  Python-3.10.19 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 186 layers, 2,684,758 parameters, 0 gradients, 6.8 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 4/4 [00:00<00:00,  8.17it/s]


                   all         53         95      0.444      0.586      0.511      0.309
                   cat         42         54      0.433      0.519      0.483      0.273
                   dog         34         41      0.455      0.653      0.538      0.344
Speed: 0.5ms preprocess, 2.3ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs\train\cats_dogs_v12


## 3. Eğitim Sonuçlarının Görselleştirilmesi (Loss & mAP)


In [5]:
# Eğitim çıktı klasörünü al
run_dir = Path(results.save_dir)
print("Run directory:", run_dir)

# YOLO'nun ürettiği results.png'yi okuyup gösterelim
img = cv2.imread(str(run_dir / "results.png"))
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

plt.figure(figsize=(10, 6))
plt.imshow(img)
plt.axis("off")
plt.title("Eğitim Sonuç Grafikleri (Loss & mAP)")
plt.show()


Run directory: runs\train\cats_dogs_v12


<Figure size 1000x600 with 1 Axes>

In [6]:
metrics = model.val(data=str(DATA_YAML))
metrics.results_dict


Ultralytics 8.3.0  Python-3.10.19 torch-2.5.1+cu121 CUDA:0 (NVIDIA GeForce RTX 4060 Laptop GPU, 8188MiB)
Model summary (fused): 186 layers, 2,684,758 parameters, 0 gradients, 6.8 GFLOPs


val: Scanning C:\Users\ibrah\Documents\BLG407_YOLOv8_Proje\dataset\valid\labels.cache... 53 images, 0 backgrounds, 0 corrupt: 100%|██████████| 53/53 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 7/7 [00:04<00:00,  1.55it/s]


                   all         53         95      0.444      0.585       0.51      0.308
                   cat         42         54      0.433      0.519      0.481      0.272
                   dog         34         41      0.455      0.651      0.538      0.344
Speed: 1.1ms preprocess, 7.4ms inference, 0.0ms loss, 1.2ms postprocess per image
Results saved to runs\train\cats_dogs_v122


{'metrics/precision(B)': 0.44375161669771335,
 'metrics/recall(B)': 0.5848372552567078,
 'metrics/mAP50(B)': 0.5095558274484121,
 'metrics/mAP50-95(B)': 0.30794513924594835,
 'fitness': 0.3281062080661947}

## 4. Doğrulama (Validation) ve Test Görselinde Tahmin


In [7]:
# Test görsellerinden bir tane seçelim
test_images_dir = PROJECT_DIR / "dataset" / "test" / "images"
sample_image = test_images_dir / os.listdir(test_images_dir)[0]

print("Örnek test görseli:", sample_image)

# Model ile tahmin alalım
results_infer = model(str(sample_image))

for r in results_infer:
    im = r.plot()  # BGR formatında çıktı
    im_rgb = cv2.cvtColor(im, cv2.COLOR_BGR2RGB)
    plt.figure(figsize=(6, 6))
    plt.imshow(im_rgb)
    plt.axis("off")
    plt.title("Örnek Tahmin")
    plt.show()


Örnek test görseli: C:\Users\ibrah\Documents\BLG407_YOLOv8_Proje\dataset\test\images\cats_118_jpg.rf.a1fc27f83d5504a76274b6a99808ecf7.jpg

image 1/1 C:\Users\ibrah\Documents\BLG407_YOLOv8_Proje\dataset\test\images\cats_118_jpg.rf.a1fc27f83d5504a76274b6a99808ecf7.jpg: 640x640 2 cats, 8.7ms
Speed: 2.0ms preprocess, 8.7ms inference, 1.9ms postprocess per image at shape (1, 3, 640, 640)


<Figure size 600x600 with 1 Axes>

In [8]:
import shutil

best_model_src = run_dir / "weights" / "best.pt"
best_model_dst = PROJECT_DIR / "best.pt"

shutil.copy(best_model_src, best_model_dst)
print("best.pt kopyalandı ->", best_model_dst)


best.pt kopyalandı -> C:\Users\ibrah\Documents\BLG407_YOLOv8_Proje\best.pt
